In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
print(os.listdir("../input"))
print(os.listdir("../input/eyn-original/"))

['eyn-original', 'ey-nextwave', 'ndscindexes']
['__notebook__.ipynb', 'train_targets.npy', 'submission.csv', '__output__.json', 'custom.css', 'train_targets_inside.npy', '__results__.html', 'train_data.npy', 'trn_index_list.npy', 'val_index_list.npy', 'test_data.npy', 'test_ids.npy']


In [2]:
dims = ['traj', 'seq', 'info']
info_col = ['t_entry','t_exit','x_entry','y_entry','x_exit','y_exit','vmax','vmin','vmean','tid_0','tid_1']

In [3]:
train_data = np.load("../input/eyn-original/train_data.npy")
print(train_data.shape)
train_data = np.moveaxis(train_data,0,1)  # should be done on pandas
index = pd.MultiIndex.from_product([range(s)for s in train_data.shape], names=dims)
df_train = pd.DataFrame({'A': train_data.flatten()}, index=index)['A']
df_train = df_train.unstack(level='info').swaplevel().sort_index()
df_train.columns = info_col
df_train[['tid_0','tid_1']] = df_train[['tid_0','tid_1']].astype('category')
df_train.head(10)

(134063, 21, 11)


t_entry    t_exit   x_entry   y_entry  ...    vmin  vmean  tid_0  tid_1
seq traj                                          ...                             
0   0     0.004194  0.030917 -1.595688 -0.429282  ...     NaN    NaN    5.0    1.0
    1    -0.036417 -0.036417 -1.599250 -0.466794  ...     NaN    NaN    4.0    1.0
    2    -0.670278 -0.637694 -1.602144 -0.533842  ...     NaN    NaN    3.0    1.0
    3    -0.710778 -0.694306 -1.603369 -0.546593  ...     NaN    NaN    2.0    1.0
    4    -0.765722 -0.757167 -1.696432 -0.835619  ...     NaN    NaN    1.0    1.0
    5    -0.792472 -0.785778 -0.988774  1.449250  ...     NaN    NaN    0.0    1.0
    6          NaN       NaN       NaN       NaN  ...     NaN    NaN    NaN    NaN
    7          NaN       NaN       NaN       NaN  ...     NaN    NaN    NaN    NaN
    8          NaN       NaN       NaN       NaN  ...     NaN    NaN    NaN    NaN
    9          NaN       NaN       NaN       NaN  ...     NaN    NaN    NaN    NaN

[10 rows x 11 columns]

In [4]:
test_data = np.load("../input/eyn-original/test_data.npy")
print(test_data.shape)
test_data = np.moveaxis(test_data,0,1)  # should be done on pandas
index = pd.MultiIndex.from_product([range(s)for s in test_data.shape], names=dims)
df_test = pd.DataFrame({'A': test_data.flatten()}, index=index)['A']
df_test = df_test.unstack(level='info').swaplevel().sort_index()
df_test.columns = info_col
df_test[['tid_0','tid_1']] = df_test[['tid_0','tid_1']].astype('category')
df_test.head(10)

(33515, 21, 11)


t_entry    t_exit   x_entry   y_entry  ...    vmin  vmean  tid_0  tid_1
seq traj                                          ...                             
0   0     0.005889  0.017556  1.221697  0.940063  ...     NaN    NaN    5.0   31.0
    1    -0.157417 -0.127972  1.248303  1.254617  ...     NaN    NaN    4.0   31.0
    2    -0.242583 -0.176361  0.285834  0.254906  ...     NaN    NaN    3.0   31.0
    3    -0.263972 -0.263972  1.229712  0.953644  ...     0.0    0.0    2.0   31.0
    4    -0.327861 -0.316194  1.251197  1.406247  ...     NaN    NaN    0.0   31.0
    5          NaN       NaN       NaN       NaN  ...     NaN    NaN    NaN    NaN
    6          NaN       NaN       NaN       NaN  ...     NaN    NaN    NaN    NaN
    7          NaN       NaN       NaN       NaN  ...     NaN    NaN    NaN    NaN
    8          NaN       NaN       NaN       NaN  ...     NaN    NaN    NaN    NaN
    9          NaN       NaN       NaN       NaN  ...     NaN    NaN    NaN    NaN

[10 rows x 11 columns]

In [5]:
df_train.to_pickle("df_train.pickle")
df_test.to_pickle("df_test.pickle")

In [6]:
# to load
df_train = pd.read_pickle("df_train.pickle")
df_test = pd.read_pickle("df_test.pickle")
print(df_train.shape, df_test.shape)

(2815323, 11) (703815, 11)


# BASELINE SUBMISSION

In [7]:
x_min, x_max = -1., 1.
y_min, y_max = -.3, .3

test_ids = np.load("../input/eyn-original/test_ids.npy")
df_test_notnull = df_test[df_test['t_entry'].notnull()]
df_test_1st_traj_only = df_test_notnull[df_test_notnull['x_exit'].isnull()]

# helper function to determine if point is inside
def is_inside(arr_x, arr_y):
    return ((arr_x > x_min) & 
            (arr_x < x_max) & 
            (arr_y > y_min) & 
            (arr_y < y_max)).astype(float)

df_submit = pd.DataFrame()
df_submit["id"] = test_ids
df_submit['target'] = is_inside(np.array(df_test_1st_traj_only['x_entry']),
                                np.array(df_test_1st_traj_only['y_entry']))
df_submit.to_csv('submission.csv', index=False)
df_submit.head()

,id,target
0,traj_00032f51796fd5437b238e3a9823d13d_31_5,0.0
1,traj_000479418b5561ab694a2870cc04fd43_25_10,0.0
2,traj_000506a39775e5bca661ac80e3f466eb_29_5,1.0
3,traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4,0.0
4,traj_00063a4f6c12e1e4de7d876580620667_3_4,0.0


In [8]:
df_submit.tail()

,id,target
33510,traj_ffe98f6e0adf12f9c7b51c4e9607a87a_15_13,0.0
33511,traj_fff607ecd3f8d3dcb65791e8b4c22a5f_3_25,1.0
33512,traj_fff813b56230c2f026f783f5b9f9ca90_19_0,0.0
33513,traj_fff9400843a88c3bfe52e7ce8bf97316_19_17,0.0
33514,traj_fff9552047b095e8242b4913f3289a26_25_7,0.0


In [9]:
# to document: specifications for all if not clear enough
# might not care to do: make the pivot table to 3D array faster - omg need to do this